In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import missingno as msno
from plotnine import *
from jupyterthemes import jtplot
jtplot.style(theme='gruvboxd', context='paper', spines=True, 
             ticks=True, grid=True, gridlines='dotted', figsize=(6,4.5))

In [82]:
# decision Tree
train=pd.read_csv("data/train.csv")
test=pd.read_csv("data/test.csv")


In [83]:
train.describe()
train.isnull().sum()



PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [84]:
train['Age_mean']=train['Age'] # 파생변수 = train 모델 | test 모델도 적용가능
train['Age_mean'].fillna(train['Age_mean'].mean(),inplace=True)
test['Age_mean']=test['Age'] # 파생변수 = train 모델 | test 모델도 적용가능
test['Age_mean'].fillna(test['Age_mean'].mean(),inplace=True) # null값 삭제되고 평균값으로 도출

In [85]:
test['Age_mean'].isnull().sum()


0

In [86]:
train['Age_mean'].mean()
print(train['Age_mean'].mean())

29.699117647058763


In [87]:
# 함수 // 레이블 인코더 & 핫 인코딩 
train['Sex'].value_counts()
train['Gender']=(train['Sex']=='famale')
test['Gender']=(train['Sex']=='famale')
# 문자형 => bullin 형 변환


In [88]:
train['Embarked'].value_counts()
test['Embarked'].value_counts()
# value_counts 하면 NaN 값 안나오므로 유무 확인 직접적 불가능
# NaN 확인하려면 isnull().sum()값 확인 
train['Embarked'].isnull().sum()
test['Embarked'].isnull().sum()
print(train['Embarked'].isnull().sum())
print(test['Embarked'].isnull().sum())

2
0


In [89]:
# 'Embarked' 이가 s,c,q 문자형 => 수치형 변환 필요 // ohe 필요
train['Embarked_S']=train['Embarked']=='S' 
train['Embarked_C']=train['Embarked']=='C' 
train['Embarked_Q']=train['Embarked']=='Q' 
# S:100, C:010, Q:001
# 속성 값이 s 와 같으면 true 반환 (Bullin형)

In [90]:
train['Embarked'].value_counts()
train['Embarked_S'].sum()

644

In [91]:
train[['Embarked_S','Embarked_C','Embarked_Q']] # OHE

,Embarked_S,Embarked_C,Embarked_Q
0,True,False,False
1,False,True,False
2,True,False,False
3,True,False,False
4,True,False,False
5,False,False,True
6,True,False,False
7,True,False,False
8,True,False,False
9,False,True,False


In [92]:
test['Embarked_S']=test['Embarked']=='S' 
test['Embarked_C']=test['Embarked']=='C' 
test['Embarked_Q']=test['Embarked']=='Q' 

In [93]:
train['FamilySize']=train['SibSp']+train['Parch']+1 # 1: 나까지 포함
# 형제자매 와 사촌의 수 칼럼 + 1 => 전체가족의 수 개체 생성 
# 부가적 파생변수 칼럼 생성하여라 (tip)
train['FamilySize'].value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: FamilySize, dtype: int64

In [94]:
# 타이타닉 데이터 .. '나'는 없다 
# 대가족 L(5명) 중가족 M(2~4명) 소가족 S(1명) 
# (구간화하여 => 모델생성 // 파생변수 생성 가능 )
train['Family']=train['FamilySize']
# train['Family']
train.loc[train['FamilySize']==1,'Family']='S'
# 해당 패밀리 군 개체수가 1 인 소가족 family 열의 값 도출 및 참조
train.loc[(train['FamilySize']>1)&(train['FamilySize']<5),'Family']='M'
train.loc[train['FamilySize']>4,'Family']='L'
train[['Family','FamilySize']]


,Family,FamilySize
0,M,2
1,M,2
2,S,1
3,M,2
4,S,1
5,S,1
6,S,1
7,L,5
8,M,3
9,M,2


In [95]:
train['Family_S']=train['Family']=='S'
train['Family_M']=train['Family']=='M'
train['Family_L']=train['Family']=='L'
train[['Family_S','Family_M','Family_L','Family','FamilySize']] 
# OHE also

,Family_S,Family_M,Family_L,Family,FamilySize
0,False,True,False,M,2
1,False,True,False,M,2
2,True,False,False,S,1
3,False,True,False,M,2
4,True,False,False,S,1
5,True,False,False,S,1
6,True,False,False,S,1
7,False,False,True,L,5
8,False,True,False,M,3
9,False,True,False,M,2


In [96]:
test['FamilySize']=test['SibSp']+test['Parch']+1 # 1: 나까지 포함
test['Family']=test['FamilySize']
test.loc[test['FamilySize']==1,'Family']='S'
test.loc[(test['FamilySize']>1)&(test['FamilySize']<5),'Family']='M'
test.loc[test['FamilySize']>4,'Family']='L'
test[['Family','FamilySize']]
test['Family_S']=test['Family']=='S'
test['Family_M']=test['Family']=='M'
test['Family_L']=test['Family']=='L'
test[['Family_S','Family_M','Family_L','Family','FamilySize']] 

,Family_S,Family_M,Family_L,Family,FamilySize
0,True,False,False,S,1
1,False,True,False,M,2
2,True,False,False,S,1
3,True,False,False,S,1
4,False,True,False,M,3
5,True,False,False,S,1
6,True,False,False,S,1
7,False,True,False,M,3
8,True,False,False,S,1
9,False,True,False,M,3


In [97]:
train.head()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Age_mean,Gender,Embarked_S,Embarked_C,Embarked_Q,FamilySize,Family,Family_S,Family_M,Family_L
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,...,34.5,False,False,False,True,1,S,True,False,False
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,...,47.0,False,True,False,False,2,M,False,True,False
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,...,62.0,False,False,False,True,1,S,True,False,False
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,...,27.0,False,True,False,False,1,S,True,False,False
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,...,22.0,False,True,False,False,3,M,False,True,False


In [98]:
# Pclass[ 수치형 ] => 범주형
train['Pclass'].dtypes

dtype('int64')

In [99]:
train['Pclass']=train['Pclass'].astype('category')
train['Pclass'].dtypes

CategoricalDtype(categories=[1, 2, 3], ordered=False)

In [121]:
# FeatureName
fn=['Gender', 'Age_mean', 
    'Embarked_S','Embarked_C','Embarked_Q',
    'Family_S','Family_M','Family_L']
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 22 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null category
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Age_mean       891 non-null float64
Gender         891 non-null bool
Embarked_S     891 non-null bool
Embarked_C     891 non-null bool
Embarked_Q     891 non-null bool
FamilySize     891 non-null int64
Family         891 non-null object
Family_S       891 non-null bool
Family_M       891 non-null bool
Family_L       891 non-null bool
dtypes: bool(7), category(1), float64(3), int64(5), object(6)
memory usage: 104.6+ KB


In [122]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 22 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
Age_mean       418 non-null float64
Gender         418 non-null bool
Embarked_S     418 non-null bool
Embarked_C     418 non-null bool
Embarked_Q     418 non-null bool
FamilySize     418 non-null int64
Family         418 non-null object
Family_S       418 non-null bool
Family_M       418 non-null bool
Family_L       418 non-null bool
Survived       418 non-null int64
dtypes: bool(7), float64(3), int64(6), object(6)
memory usage: 51.9+ KB


In [114]:
# train data 의 입력과 출력
# 예측 대상 y 
X_train=train[fn]
# X_train.head()
y_label=train['Survived']
X_test=test[fn] 
# 모델링 [Decision Tree]
from sklearn.tree import DecisionTreeClassifier
# tree 서브 패키지 # Regressort [회귀 - 예측] 모델 도 적용가능
model=DecisionTreeClassifier(max_depth=10) #default - 3 or 5 // 킬람 3개만 바
# 변수가 많을 시 연산량이 방대해지므로 칼럼수를 줄여준다 
# 객체 생성(model)
model # criterion = 'gini' # 불순도 

## 실제 모델 생성 ## 
model.fit(X_train, y_label) 
# 모델 트리 생성 완료
# => 다수의 트리 : 숲 (Random Forest)

# 엔트로피 계산 [분류이전] - (각각의 컬럼으로 나누었을때)
# 얻을 수 있는 IG [분류한 이후]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [115]:
from sklearn.model_selection import KFold, cross_val_score
kfold=KFold(n_splits=10, shuffle=True, random_state=2019)
kfold

KFold(n_splits=10, random_state=2019, shuffle=True)

In [116]:
score=cross_val_score(model, X_train, y_label, cv=kfold, 
                      scoring='accuracy') 
print(score)
# (model 객체, x, y, cv=kfold)
round(np.mean(score)*100,2)

[0.6        0.5505618  0.70786517 0.60674157 0.73033708 0.71910112
 0.52808989 0.64044944 0.76404494 0.5505618 ]


63.98

In [117]:
prediction=model.predict(X_test)
prediction.shape
# 0 - 죽음 # 1 - 삶
# 열개의 컬럼 중 90건씩 들어가있는 데이터에 대해서 첫번째 / test
# 나머지 810에 대해서 모델만들고 train data로 이용한다 


(418,)

In [118]:
test['Survived']=prediction
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age_mean', 'Gender',
       'Embarked_S', 'Embarked_C', 'Embarked_Q', 'FamilySize', 'Family',
       'Family_S', 'Family_M', 'Family_L', 'Survived'],
      dtype='object')

In [119]:
submission=test[['PassengerId','Survived']]
submission.to_csv("submit.csv", index=False)


In [ ]:
# 윤동주 시들에 대한 제목데이터 추출 
# or wikipedia total data 끌어오기 
# 음성 - 텍스트 - 영뮨 // 해당텍스트에 대한 적합한 대답 찾기 RNN / Wordembed
# RNN 기반( 순서 따져 )
# 유사도 기반 ( 순서 안따져 )
# 파이썬 코딩중에 변수 

# 정규 표현법 
# \w+\s*\w*\s*\=*\s*\"*\d*\"*\;*
# [\w\d\"\s\=\;]+

 # 코사인 유사도 
# 네이버 파파고 API 따와서 라이브러리 활용 

In [5]:
# urllib:url을 다루는 모듈의 집합 
# urllib.request: 웹 데이터 접근
# 쿠키 : 서버쪽의 클라이언트(접속자)에 대한 남겨진 정보가 남는다 // 클라이언트 측에 남긴다
# 세션 : 서버쪽에 저장되는 내정보 
# 접속시에는 쿠키 // 통신시에는 세션을 사용해서 로그인상태를 유지한다 
import urllib.request
url="https://www.google.co.kr/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png"
sname="./data/data/hojin.png"
urllib.request.urlretrieve(url, sname)
print("저장됐습니다")
# 데이터 읽어서 저장 작업

FileNotFoundError: [Errno 2] No such file or directory: './data/data/data/hojin.png'

In [ ]:
import urllib.request
url="https://www.google.co.kr/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png"
sname="test.png"
mem=urllib.request.urlopen(url).read()
mem
with open(sname, mode="wb" ) as f: # test.png
    f.write(mem)
    print("이미지가 또 저장됐습니다.")